In [1]:
#!pip install stable-baselines3[extra]

  Created wheel for gym: filename=gym-0.18.3-py3-none-any.whl size=1657517 sha256=6712e73ce621a6275dcf43d109109858cec2cdcc0bf4d25506343d78a587193f
  Stored in directory: c:\users\jesse\appdata\local\pip\cache\wheels\b3\03\54\9141c232861b89be935b37bdde0ea5ab472f5e18fc20623aed
Successfully built gym


In [1]:
# import dependencies

import os 
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

In [2]:
# load environment 

environment_name = 'CartPole-v0'
env = gym.make(environment_name)

In [4]:
episodes = 5

for episode in range(1, episodes + 1):
  state = env.reset()
  done = False
  score = 0 
  while not done:
    env.render()
    action = env.action_space.sample()
    n_state, reward, done, info = env.step(action)
    score += reward
  print('Episod: {} Score: {}'.format(episode, score))

env.close()

Episod: 1 Score: 24.0
Episod: 2 Score: 12.0
Episod: 3 Score: 18.0
Episod: 4 Score: 21.0
Episod: 5 Score: 42.0


In [12]:
env.action_space

Discrete(2)

In [10]:
# understanding the environment

env.action_space.sample()

1

In [11]:
env.observation_space

Box(-3.4028234663852886e+38, 3.4028234663852886e+38, (4,), float32)

In [6]:
env.observation_space.sample()

array([-4.5120394e-01,  7.3608409e+37,  3.4732372e-01, -6.2629789e+37],
      dtype=float32)

# trainning 
types of algorithms 
model vs model free: learning based on predictsion of next state/reward or real samples. 

stable baselines focuses on model free algorithms

Choosing algorithms 
there are a number of algorithms available through stable baseliness as shown to the right. we can easily switch between each of these. 
certain algorithms will perform better for certain environements. Often it helps to review literature in order to determine the best approach. 


understanding trainning metrics 
evaluation metrics: 
ep_len_mean, eo_rew_mean

time metrics:
fps, iterations, time_elapsed, total_timestpes

loss metrics:
entropy_loss, policy_loss, value_loss

other metrics:
explained_variance, learning_rate, n_updates

In [14]:
# make your directories first
log_path = os.path.join('Trainning', 'Logs')


In [15]:
log_path

'Trainning\\Logs'

In [16]:
env = gym.make(environment_name)
env = DummyVecEnv([lambda: env])
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device


In [19]:
model.learn(total_timesteps=20000)

Logging to Trainning\Logs\PPO_2
-----------------------------
| time/              |      |
|    fps             | 2793 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
------------------------------------------
| time/                   |              |
|    fps                  | 1578         |
|    iterations           | 2            |
|    time_elapsed         | 2            |
|    total_timesteps      | 4096         |
| train/                  |              |
|    approx_kl            | 0.0035090137 |
|    clip_fraction        | 0.0115       |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.573       |
|    explained_variance   | 0.305        |
|    learning_rate        | 0.0003       |
|    loss                 | 17.5         |
|    n_updates            | 110          |
|    policy_gradient_loss | -0.000822    |
|    value_loss           | 108          |
---------------------------

In [20]:
# save and reload mdoel 
PPO_Path = os.path.join('Trainning', "Saved Models", 'PPO_Model_Cartpole')

In [21]:
model.save(PPO_Path)

In [22]:
del model

In [23]:
model = PPO.load(PPO_Path, env = env)

In [24]:
model.learn(total_timesteps=1000)

Logging to Trainning\Logs\PPO_3
-----------------------------
| time/              |      |
|    fps             | 2750 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------


# evaluation 
There are a number of metrics available from the modesl when trained but two core values that you should pay attention to:
ep_len_mean: on average how long a particular episode lasted before done
ep_rew_mean: the average rewardd that the agent accumulated per episode

Monitoring in tensorboard 
you are able to review evaluation, time and trainning modetrics from within tensorboard
in order to do so you must specify a logging directory when you initialise your model 


In [26]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

C:\ProgramData\Anaconda3\lib\site-packages\stable_baselines3\common\evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


(200.0, 0.0)

In [30]:
action, _ = model.predict(obs)

In [32]:
# test model 
episodes = 5

for episode in range(1, episodes + 1):
  obs = env.reset()
  done = False
  score = 0 
  while not done:
    env.render()
    action, _  = model.predict(obs) # now we are using model here 
    obs, reward, done, info = env.step(action)
    score += reward
  print('Episod: {} Score: {}'.format(episode, score))

env.close()

Episod: 1 Score: [200.]
Episod: 2 Score: [200.]
Episod: 3 Score: [200.]
Episod: 4 Score: [200.]
Episod: 5 Score: [200.]


In [33]:
obs = env.reset()

In [47]:
action, _ = model.predict(obs)

In [46]:
env.action_space.sample()

1

In [48]:
env.step(action)

(array([[-0.03568621,  0.18240808, -0.00432363, -0.3119863 ]],
       dtype=float32),
 array([1.], dtype=float32),
 array([False]),
 [{}])

In [49]:
# viewing logs in tensorboard 
trainning_log_path = os.path.join(log_path, 'PPO_2')

In [50]:
trainning_log_path

'Trainning\\Logs\\PPO_2'

In [55]:
# this doesn't work for some reason, just run it in command prompt and use the follow code 
!python -m tensorboard.main --logdir=trainning_log_path

#!tensorboard --logdir={trainning_log_path}

^C


# core metrics to look at 
* Average reward
* average episode length 

trainning strategies 
* train for longer 
* hyperparameter tuning 
* try different algorithms 


# applying callbacks 
you can leverage callback functions as part of stable baselines to log out data or save the model under certain conditions 
# using different algorithms 
stable baselines comes pre-packaged with a number of different algorithms that can be used to train you agent 

In [56]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [57]:
save_path = os.path.join('Trainning', 'Saved Models')

In [63]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200, verbose = 1)
eval_callback = EvalCallback(env,
                            callback_on_new_best=stop_callback,
                            eval_freq=10000,
                            best_model_save_path=save_path,
                            verbose=1)

In [64]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path)


Using cpu device


In [65]:
model.learn(total_timesteps=20000, callback=eval_callback)

Logging to Trainning\Logs\PPO_4
-----------------------------
| time/              |      |
|    fps             | 3095 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1847        |
|    iterations           | 2           |
|    time_elapsed         | 2           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008498399 |
|    clip_fraction        | 0.0812      |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.687      |
|    explained_variance   | -0.000221   |
|    learning_rate        | 0.0003      |
|    loss                 | 7.15        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0119     |
|    value_loss           | 44.8        |
-----------------------------------------
--

In [67]:
# change policies
net_arch = [dict(pi=[128,128,128,128], vf=[128,128,128,128])]

In [69]:
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_path, policy_kwargs={'net_arch': net_arch})


Using cpu device


In [70]:
model.learn(total_timesteps=2000,callback=eval_callback)

Logging to Trainning\Logs\PPO_5
-----------------------------
| time/              |      |
|    fps             | 2602 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 2048 |
-----------------------------


In [71]:
# using alternate algorithm
from stable_baselines3 import DQN

In [72]:
model = DQN('MlpPolicy', env, verbose=1, tensorboard_log=log_path)

Using cpu device


In [73]:
model.learn(total_timesteps=20000)

  | 652      |
|    fps              | 13305    |
|    time_elapsed     | 1        |
|    total timesteps  | 14501    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 656      |
|    fps              | 13311    |
|    time_elapsed     | 1        |
|    total timesteps  | 14614    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 660      |
|    fps              | 13322    |
|    time_elapsed     | 1        |
|    total timesteps  | 14706    |
----------------------------------
----------------------------------
| rollout/            |          |
|    exploration rate | 0.05     |
| time/               |          |
|    episodes         | 664      |
|    fps              | 13314    |
|    